In [8]:
import numpy as np
import pandas as pd
from time import sleep

import requests

from pprint import pprint
import os

In [9]:
TOKEN =  os.getenv("AIRTABLE_TOKEN") # Usuario

BASE_ID = "appOnz4LtIdKAOYfo" # Base

TABLE_ID = "" # Tabla:

airtable_base_url = "https://api.airtable.com/v0"

# Headers
headers = {"Authorization" : f"Bearer {TOKEN}",
           "Content-Type"  : "application/json"}

In [10]:
df = pd.read_csv("Data/Meneos_Iniciales_y_finales_2023.csv")
df.head()

,Indice,Comunidad,Usuario,Medio,Enviado,Publicado,Titular,Entradilla,Votos,Clicks,Positivos,Anonimos,Negativos,Comentarios,Karma
0,0,politica,dmeijide,twitter.com,31/12/2023 12:05,31/12/2023 22:40,Yanis Varoufakis: Suspended el Apartheid Israe...,Como una vez suspendimos el Apartheid de Sudáf...,649,1237,276,373,9,57,399
1,1,actualidad,--557077--,europapress.es,31/12/2023 17:11,31/12/2023 21:25,Irán libera al español Santiago Sánchez Cogedo...,La Justicia de Irán ha puesto en libertad este...,280,1206,140,140,1,108,499
2,2,actualidad,tropezon,diariodecadiz.es,30/12/2023 23:31,31/12/2023 21:05,Estafan a un vecino de Vejer casi 10.000 euros...,Francisco Raúl Moreno Mera recibió tres mensaj...,236,4343,103,133,2,110,473
3,3,actualidad,FatherKarras,kaosenlared.net,31/12/2023 17:13,31/12/2023 20:25,Pablo González: un secuestro político a la com...,"Cuesta entender, incluso si le añadimos la eti...",573,710,232,341,17,124,486
4,4,actualidad,silver_hoodlum,twitter.com,31/12/2023 19:12,31/12/2023 19:40,"Video: El cacheo pacífico de los soldados ""Isr...",Un video vale mas que mil palabras de Netanyah...,580,4640,235,345,5,77,463


In [34]:
nombres_columnas = {"name" : "Meneame_2023",
        "description" : "Listado de meneos de la portada general de 2023",
        "fields": [
            {
                "name" : "Titular",
                "type" : "singleLineText"
            },
            {
                "name" : "Entradilla",
                "type" : "multilineText"
            },
            {
                "name" : "Comunidad",
                "type" : "singleSelect",
                "options" : {
                    "choices":[{                        
                        "name" : "politica"
                    },
                    {                        
                        "name" : "ciencia"
                    }
                    ]
                }
            },
            {
                "name" : "Medio",
                "type" : "singleLineText"
            },
            {
                "name" : "Fechas envío",
                "type" : "dateTime",
                "options": {
                    "timeZone" : "Europe/Madrid",
                    "dateFormat" : {
                        "format" : "YYYY-MM-DD",
                        "name": "iso"
                    },
                    "timeFormat" : {
                        "format": "HH:mm",
                        "name": "24hour"
                    }
                }
            },
            {
                "name" : "Fecha publicación",
                "type" : "dateTime",
                "options": {
                    "timeZone" : "Europe/Madrid",
                    "dateFormat" : {
                        "format" : "YYYY-MM-DD",
                        "name": "iso"
                    },
                    "timeFormat" : {
                        "format": "HH:mm",
                        "name": "24hour"
                    }
                }
            },
            {
                "name" : "Número de meneos",
                "type" : "number",
                "options" : {
                    "precision" : 0
                }
            },
            {
                "name" : "Número de clicks",
                "type" : "number",
                "options" : {
                    "precision" : 0
                }
            },
            {
                "name" : "Votos positivos",
                "type" : "number",
                "options" : {
                    "precision" : 0
                }
            },
            {
                "name" : "Votos anónimos",
                "type" : "number",
                "options" : {
                    "precision" : 0
                }
            },
            {
                "name" : "Votos negativos",
                "type" : "number",
                "options" : {
                    "precision" : 0
                }
            },
            {
                "name" : "Numero comentarios",
                "type" : "number",
                "options" : {
                    "precision" : 0
                }
            },
            {
                "name" : "Usuario",
                "type" : "singleLineText"
            },
            {
                "name" : "karma",
                "type" : "number",
                "options" : {
                    "precision" : 0
                }
            },            
            {
                "name" : "Delay",
                "type" : "duration",
                "options" : {
                    "durationFormat" : "h:mm"
                }
            },            
            {
                "name" : "Mes",
                "type" : "singleLineText",
            },
            {
                "name" : "Trimestre",
                "type" : "singleLineText"
            },
            {
                "name" : "Día de la semana",
                "type" : "singleLineText"
            },
            {
                "name" : "Franja horaria",
                "type" : "singleLineText"
            },
            {
                "name" : "Municipio",
                "type" : "singleLineText"
            },
            {
                "name" : "Provincia",
                "type" : "singleLineText"
            },
            {
                "name" : "Latitud",
                "type" : "number",
                "options" : {
                    "precision" : 8
                }
            },
            {
                "name" : "Longitud",
                "type" : "number",
                "options" : {
                    "precision" : 8
                }
            }
        ]
        }

In [ ]:
# tipos_airtable = {"object":"singleLineText", "int64":"number\", \"options\":{ \"precision\":2} "}
# nombres_columnas = {"description": "Listado de meneos de la portada general de 2023",
#                     "fields": [{"name" : columna, "type": tipos_airtable[str(tipo)] } for columna, tipo in zip(df.columns,df.dtypes)],
#                     "name": "MeneosInicialesFinales2023"}

In [35]:
endpoint_crea_tabla = f"https://api.airtable.com/v0/meta/bases/{BASE_ID}/tables"
    
response = requests.post(url = endpoint_crea_tabla, json= nombres_columnas, headers = headers)

print(f"endpoint: {response.url}")
print(f"response: {response.status_code}")
print("-"*120)
pprint(response.json(), sort_dicts = False)
print("-"*120)

endpoint: https://api.airtable.com/v0/meta/bases/appOnz4LtIdKAOYfo/tables
response: 200
------------------------------------------------------------------------------------------------------------------------
{'id': 'tblauhsgAn5y19nKg',
 'name': 'Meneame_2023',
 'description': 'Listado de meneos de la portada general de 2023',
 'primaryFieldId': 'fldojTWhBQqigQP7I',
 'fields': [{'type': 'singleLineText',
             'id': 'fldojTWhBQqigQP7I',
             'name': 'Titular'},
            {'type': 'multilineText',
             'id': 'fld29o2vROb2TOE97',
             'name': 'Entradilla'},
            {'type': 'singleSelect',
             'options': {'choices': [{'id': 'sellqAPIpBZ4p1w7Z',
                                      'name': 'politica',
                                      'color': 'grayLight2'},
                                     {'id': 'selCOclNkw2UoDzMX',
                                      'name': 'ciencia',
                                      'color': 'grayLight2'}]